In [ ]:
REQUEST_ID = 1013
AOI = 'POLYGON ((-91.93050383241031 34.50910284968832, -91.92844389588556 34.61032319084964, -92.074699389025 34.61145344841528, -92.07538603453924 34.51362929274102, -92.00328825622739 34.50966866851616, -92.00328825622739 34.50966866851616, -91.93050383241031 34.50910284968832))'
START_DATE = '2020-07-01'
END_DATE = '2020-08-18'

In [ ]:
import os
import json
import time
import cv2
import rasterio
import pandas as pd
import numpy as np
import geopandas as gpd
import rasterio.mask
import shapely

from tqdm.notebook import tqdm
from os.path import join, basename, split
from scipy.ndimage import rotate
from rasterio.features import rasterize, shapes
from rasterio.merge import merge
from rasterio.plot import reshape_as_raster
from shapely.geometry import Polygon, shape, LinearRing
import shapely.wkt
from pathlib import Path
from datetime import datetime

from sklearn.decomposition import LatentDirichletAllocation
from sklearn.cluster import KMeans
from sklearn.neighbors import KernelDensity, NearestNeighbors
import rasterio.mask as riomask

from code.download.utils import get_tiles, _check_folder, check_nodata, get_min_clouds
from code.download.load_tiles import load_images
from code.index_research import calculate_ndmi, calculate_ndvi

import warnings
warnings.filterwarnings('ignore')
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [ ]:
default_crs = 'EPSG:4326'

polygon = shapely.wkt.loads(AOI)
aoi_filename = f"{time.time()}_aoi.geojson"
gpd.GeoDataFrame(gpd.GeoSeries([polygon]), columns=["geometry"]).to_file(aoi_filename, driver="GeoJSON")

In [ ]:
NB_USER = os.getenv('NB_USER')
BASE = os.getcwd()
DATA_DIR = f"/home/{NB_USER}/work"

API_KEY = os.path.join(DATA_DIR, ".secret/sentinel2_google_api_key.json")
sentinel_tiles_path = os.path.join(BASE, "notebooks/pbdnn/sentinel2grid.geojson")
LOAD_DIR = os.path.join(DATA_DIR, "satellite_imagery")
RESULTS_DIR = os.path.join(DATA_DIR, f"results/dd/moisture_content/{REQUEST_ID}")
NDMI_PATH = os.path.join(BASE, f'data/rasters/{REQUEST_ID}_ndmi.tif')
NDVI_PATH = os.path.join(BASE, f'data/rasters/{REQUEST_ID}_ndvi.tif')
SENTINEL_TILES_PATH = os.path.join(BASE, "sentinel2grid.geojson")
BANDS = {'B04', 'B08', 'B12', 'TCI'}
CONSTRAINTS = {'NODATA_PIXEL_PERCENTAGE': 15.0, 'CLOUDY_PIXEL_PERCENTAGE': 10.0, }
PRODUCT_TYPE = 'L2A'

In [ ]:
date_tile_info = get_tiles(aoi_filename, SENTINEL_TILES_PATH)
loadings = load_images(API_KEY, date_tile_info.tileID.values, START_DATE, END_DATE, LOAD_DIR, BANDS, CONSTRAINTS, PRODUCT_TYPE)
checked = check_nodata(loadings, PRODUCT_TYPE)

try:
    checked = get_min_clouds(checked)
except Exception:
    print(f'No clean raster found for period from {START_DATE} to {END_DATE}, skipping')

for i, tile in date_tile_info.iterrows():
    try:
        tile_folder = Path(checked[tile.tileID])
        print(f'filtered: {tile_folder}')
    except Exception as ex:
        print(ex)
        continue

    b04_tile = [os.path.join(tile_folder, filename) for filename in os.listdir(tile_folder) if 'B04_10m.jp2' in filename][0]
    b08_tile = [os.path.join(tile_folder, filename) for filename in os.listdir(tile_folder) if 'B08_10m.jp2' in filename][0]
    b12_tile = [os.path.join(tile_folder, filename) for filename in os.listdir(tile_folder) if 'B12_20m.jp2' in filename][0]
    tci_tile = [os.path.join(tile_folder, filename) for filename in os.listdir(tile_folder) if 'TCI_10m.jp2' in filename][0]

In [ ]:
os.makedirs(RESULTS_DIR, exist_ok=True)
calculate_ndmi(b08_tile, b12_tile, out_path=NDMI_PATH, nodata=np.nan)
calculate_ndvi(b04_tile, b08_tile, out_path=NDVI_PATH, nodata=np.nan)

In [ ]:
aoi = gpd.read_file(aoi_filename)

with rasterio.open(tci_tile) as src:
    tci_image, tfs = riomask.mask(
        src, aoi.to_crs(src.crs).geometry, all_touched=False, crop=True)
    
with rasterio.open(NDMI_PATH) as src:
    ndmi, tfs = riomask.mask(
        src, aoi.to_crs(src.crs).geometry, all_touched=False, crop=True)
    meta = src.meta
    meta['transform'] = tfs
    meta['width'] = ndmi.shape[-1]
    meta['height'] = ndmi.shape[-2]
    
with rasterio.open(NDVI_PATH) as src:
    ndvi, _ = riomask.mask(
        src, aoi.to_crs(src.crs).geometry, all_touched=False, crop=True)

In [ ]:
# class_name -> [[(ndmi_range), (ndvi_range)], [(ndmi_range), (ndvi_range)], ... ]

class_names = {
    
    "no water stress": [
        [(-0.6, -0.4), (0.1, 0.2)],
        [(-0.4, -0.2), (0.1, 0.3)],
        [(-0.2, 0.0), (0.1, 0.3)],
        [(0.0, 0.2), (0.1, 0.4)],
        [(0.2, 0.4), (0.1, 0.4)],
        [(0.4, 0.6), (0.7, 1.0)],
        [(0.6, 0.8), (0.6, 1.0)],
        [(0.8, 1.0), (0.5, 1.0)],
        
    ],
    
    "low water stress": [
        [(-0.6, -0.4), (0.2, 0.3)],
        [(-0.4, -0.2), (0.3, 0.4)],
        [(-0.2, 0.0), (0.3, 0.5)],
        [(0.0, 0.2), (0.4, 0.7)],
        [(0.2, 0.4), (0.4, 0.5)],
        [(0.4, 0.6), (0.4, 0.7)],
        [(0.6, 0.8), (0.3, 0.6)],
        [(0.8, 1.0), (0.1, 0.5)],
    ],
    
    "high water stress" : [
        [(-0.6, -0.4), (0.3, 0.6)],
        [(-0.4, -0.2), (0.4, 0.6)],
        [(-0.2, 0.0), (0.5, 0.7)],
        [(0.0, 0.2), (0.7, 0.9)],
        [(0.2, 0.4), (0.5, 0.9)],
        [(0.4, 0.6), (0.1, 0.4)],
        [(0.6, 0.8), (0.1, 0.3)],
    ],
    
    "drought": [
        [(-0.4, -0.2), (0.6, 1.0)],
        [(-0.6, -0.4), (0.6, 1.0)],
        [(-0.2, 0.0), (0.7, 1.0)],
        [(0.0, 0.2), (0.9, 1.0)],
        [(0.2, 0.4), (0.9, 1.0)]
    ]
}

NUM_CLASSES = len(class_names)
arr = np.array(range(0, NUM_CLASSES)) / NUM_CLASSES

colors = [
    (138, 206, 126),
    (48, 145, 67),
    (255, 218, 102),
    (182, 10, 28),
]

labels = []

mask = np.zeros((ndmi[0].shape[-2], ndmi[0].shape[-1], 3)).astype(np.uint8)
for idx, (name, values) in enumerate(class_names.items()):
    class_area = 0
    for pix_vals in values:
        
        ndmi_pix, ndvi_pix = pix_vals
        class_area += np.where(((ndmi[0] >= ndmi_pix[0])&(ndmi[0] <= ndmi_pix[1])) & ((ndvi[0] >= ndvi_pix[0])&(ndvi[0] <= ndvi_pix[1])), 1, 0).sum() / 10**4 
        mask[((ndmi[0] >= ndmi_pix[0])&(ndmi[0] <= ndmi_pix[1])) & ((ndvi[0] >= ndvi_pix[0])&(ndvi[0] <= ndvi_pix[1]))] = colors[idx]

    labels.append({
        "color": ",".join(list(map(lambda x: str(int(x)), colors[idx]))),
        "name": name,
        "area": round(class_area, 3)
    })

In [ ]:
labels = json.dumps(labels)
mask = mask.astype(np.float32)
labels

In [ ]:
meta.update(
    count=3,
    nodata=0,
    compress='lzw',
    photometric='RGB'
)

result_name = f"{REQUEST_ID}_{START_DATE}_{END_DATE}.tif"
colored_tif = os.path.join(RESULTS_DIR, result_name)

with rasterio.open(colored_tif, 'w', **meta) as dst:
    dst.update_tags(start_date=START_DATE, 
                    end_date=END_DATE, 
                    request_id=REQUEST_ID,
                    labels=labels,
                    name='Moisture content')

    for i in range(mask.shape[-1]):
        dst.write(mask[:,:,i], indexes=i+1)

In [ ]:
os.remove(aoi_filename)

In [ ]:
json_classes = {i: x for i, x in enumerate(class_names.keys())}

In [ ]:
if not os.path.exists('classes_labels.json'):
    with open('classes_labels.json', 'w') as file:
        json.dump(json_classes, file)

In [ ]:
remove_aux_rasters = True

if remove_aux_rasters:
    
    try:
        os.remove(NDMI_PATH)
    except FileNotFoundError:
        print('No helping rasters found')
        
    try:
        os.remove(NDVI_PATH)
    except FileNotFoundError:
        print('No helping rasters found')